# Ejercicio 47
La mediana de cantidad de links internos que tienen todos los contenidos que existen.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
dfPageLinks = pd.read_csv('./WikipediaDataset/pagelinks_sample.csv', usecols = ['pl_from'])
dfContents = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["id"])

## Limpiando y preparando los datasets
Se procede a limpiar los datasets para usarlos. Al ser documentos con gran cantidad de datos, se leyeron de antemano solo las columnas imprescindibles para realizar la consigna, necesario para ahorrar memoria y aumentar performance.
Se trabaja sobre la columna de ids de ambos documentos. Se eliminan posibles filas que tengan NaN.

In [2]:
dfPageLinks.head()

,pl_from
0,4840492
1,950948
2,6418187
3,3436623
4,4713066


In [3]:
dfPageLinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000000 entries, 0 to 79999999
Data columns (total 1 columns):
 #   Column   Dtype
---  ------   -----
 0   pl_from  int64
dtypes: int64(1)
memory usage: 610.4 MB


In [4]:
dfPageLinks = dfPageLinks[dfPageLinks['pl_from'].notna()]

In [5]:
dfPageLinks["pl_from"].value_counts()

2587499    18831
2389056    17495
2389124    17349
2389118    17344
2653364    16609
           ...  
7096870        1
2900519        1
7293510        1
8568058        1
9530785        1
Name: pl_from, Length: 6051413, dtype: int64

Se añaden al dataframe la cantidad de links internos que tiene el respectivo id. 

In [6]:
dfPageLinks["cant_links_internos"] = dfPageLinks.groupby("pl_from")["pl_from"].transform('count')
dfPageLinks

,pl_from,cant_links_internos
0,4840492,16314
1,950948,1
2,6418187,41
3,3436623,1
4,4713066,498
...,...,...
79999995,5945261,89
79999996,2850153,4656
79999997,8618148,16
79999998,7113180,43


Para facilitar la forma de interpretar el merge se cambia el nombre de "pl_from" a "id_fuente"

In [7]:
dfPageLinks.rename(columns = {'pl_from': 'id_fuente'}, inplace = True)
dfPageLinks.head()

,id_fuente,cant_links_internos
0,4840492,16314
1,950948,1
2,6418187,41
3,3436623,1
4,4713066,498


Como se va a hacer un merge con el otro dataframe, es importante eliminar las filas con ids duplicados. De esta forma, a la hora de hacer el right merge se van a obtener el número de filas del right dataframe, este es el df de contents.

In [8]:
dfPageLinks.drop_duplicates(subset = ["id_fuente"], inplace = True, ignore_index = True)
dfPageLinks

,id_fuente,cant_links_internos
0,4840492,16314
1,950948,1
2,6418187,41
3,3436623,1
4,4713066,498
...,...,...
6051408,4536622,1
6051409,3739769,1
6051410,5034409,1
6051411,4911668,1


In [9]:
dfContents.head()

,id
0,5
1,7
2,10
3,15
4,17


In [10]:
dfContents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132164 entries, 0 to 4132163
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   id      int64
dtypes: int64(1)
memory usage: 31.5 MB


In [11]:
dfContents = dfContents[dfContents['id'].notna()]

También se cambia el nombre de la columna "id" a "id_fuente".

In [12]:
dfContents.rename(columns = {'id': 'id_fuente'}, inplace = True)
dfContents.head()

,id_fuente
0,5
1,7
2,10
3,15
4,17


## Desarrollo

In [13]:
dfMerge = pd.merge(dfPageLinks, dfContents, how = "right", on = ["id_fuente"])
dfMerge

,id_fuente,cant_links_internos
0,5,2089.0
1,7,463.0
2,10,1681.0
3,15,98.0
4,17,26.0
...,...,...
4132159,10010593,9.0
4132160,10010595,1.0
4132161,10010596,NaN
4132162,10010598,6.0


In [14]:
dfMerge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4132164 entries, 0 to 4132163
Data columns (total 2 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_fuente            int64  
 1   cant_links_internos  float64
dtypes: float64(1), int64(1)
memory usage: 94.6 MB


Después del merge, el dataframe resultante tiene todas las filas del dataframe contents ya que fue un right join. Sin embargo, hay ids del contents que no estaban en la lista de ids del page_links, quedando así columnas de cant_links_internos con NaN. Esto significa que los ids con NaN en cant_links_internos no tienen justamente link internos, por lo que se puede reemplazar con 0s.  

In [15]:
dfMerge['cant_links_internos'] = dfMerge['cant_links_internos'].fillna(0)
dfMerge

,id_fuente,cant_links_internos
0,5,2089.0
1,7,463.0
2,10,1681.0
3,15,98.0
4,17,26.0
...,...,...
4132159,10010593,9.0
4132160,10010595,1.0
4132161,10010596,0.0
4132162,10010598,6.0


In [19]:
dfMerge["cant_links_internos"].value_counts()

1.0       1399093
0.0        865134
15.0        59616
14.0        59491
16.0        58970
           ...   
1335.0          1
1332.0          1
1328.0          1
1321.0          1
1889.0          1
Name: cant_links_internos, Length: 1515, dtype: int64

Ahora se puede calcular la mediana de la columna cant_links_internos, respondiendo así la consigna.

In [16]:
mediana = dfMerge["cant_links_internos"].median()
mediana

1.0

## Conclusiones
Al asumir que todos los contenidos que existen están en contents, y todos los links internos que existen están en page_links_sample, se pudo concluir que la mediana de links internos es 1. Esto sifnifica que el 50 % de cantidad de links internos está entre 0 y 1 y el otro 50 % es mayor o igual a 1. 